In [1]:
import os

In [2]:
os.chdir("../")
os.getcwd()

'/home/adam/Workspace/personal/unihack/fast-form'

In [6]:
import numpy as np
import pandas as pd

train_db = pd.read_csv("training_data/emnist-balanced-train.csv")
test_db  = pd.read_csv("training_data/emnist-balanced-test.csv")

# train model

In [7]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import model_from_json

num_classes = 47
y_train = train_db.iloc[:,0]
y_train = np_utils.to_categorical(y_train, num_classes)
print ("y_train:", y_train.shape)

x_train = train_db.iloc[:,1:]
x_train = x_train.astype('float32')
x_train /= 255
print ("x_train:",x_train.shape)

inp = Input(shape=(784,))
hidden_1 = Dense(1024, activation='relu')(inp)
dropout_1 = Dropout(0.2)(hidden_1)
out = Dense(num_classes, activation='softmax')(hidden_1) 
model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=512, nb_epoch=10,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

y_test = test_db.iloc[:,0]
y_test = np_utils.to_categorical(y_test, num_classes)
print ("y_test:", y_test.shape)

x_test = test_db.iloc[:,1:]
x_test = x_test.astype('float32')
x_test /= 255
print ("x_test:",x_train.shape)

print(model.evaluate(x_test, y_test, verbose=1)) # Evaluate the trained model on the test set!

y_train: (112799, 47)
x_train: (112799, 784)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Train on 101519 samples, validate on 11280 samples
Epoch 1/10
101519/101519 [==============================] - 9s 93us/step - loss: 1.2407 - accuracy: 0.6589 - val_loss: 0.8499 - val_accuracy: 0.7462
Epoch 2/10
101519/101519 [==============================] - 10s 96us/step - loss: 0.7157 - accuracy: 0.7868 - val_loss: 0.6585 - val_accuracy: 0.7908
Epoch 3/10
101519/101519 [==============================] - 8s 83us/step - loss: 0.5633 - accuracy: 0.8247 - val_loss: 0.5617 - val_accuracy: 0.8179
Epoch 4/10
101519/101519 [==============================] - 10s 100us/step - loss: 0.4867 - accuracy: 0.8445 - val_loss: 0.5280 - val_accuracy: 0.8251
Epoch 5/10
101519/101519 [==============================] - 8s 80us/step - loss: 0.4345 - accuracy: 0.8568 - val_loss: 0.5101 - val_accuracy: 0.8316
Epoch 6/10
101519/101519 [==============================] - 8s 80us/step - loss: 0.3969 - accuracy: 0.8672 - val_loss: 0.4897 - val_accuracy: 0.8326
Epoch 7/10
101519/101519 [=========================

In [8]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 84.86%


## model save/load

In [9]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 
# serialize model to JSON
model_json = model.to_json()
with open("model_data/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_data/model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model_data/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_data/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

accuracy: 84.86%
Saved model to disk
Loaded model from disk
accuracy: 84.86%


# predictions

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline
import PIL.Image
from IPython.display import Image

In [11]:
with open("training_data/emnist-balanced-mapping.txt") as f:
    s = f.read()[:-1] # remove last char \n
a = [l.split(" ") for l in s.split("\n")]
res_mapper = {int(l[0]): chr(int(l[1])) for l in a}
print(res_mapper)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T', 30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z', 36: 'a', 37: 'b', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'n', 44: 'q', 45: 'r', 46: 't'}


# Display data from dataset

In [205]:
type(test_db.iloc[i].values[1:][0])

numpy.int64

In [281]:
def show_letter_from_db(img):
    img = img.astype('uint8')
    img = img.reshape((28,28))
    img = img.transpose()
    img = cv2.bitwise_not(img)
    
    display(PIL.Image.fromarray(img))
    return img

In [282]:
def revert_showing(img):
    img = cv2.bitwise_not(img)
    img = img.transpose()
    img = img.ravel()
    img = img.astype('int64') 
    return img

In [283]:
def display_char(i):
    label = test_db.iloc[i].values[0]
    char = res_mapper.get(label)
    print("=============")
    print(f"label: {char}")
    
    img = test_db.iloc[i].values[1:]
    print("showing original")
    img_new=show_letter_from_db(img)
    
    img = revert_showing(img_new)
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    print("showing the one prediction run on")
    show_letter_from_db(img)

In [285]:
for i in range (3):
    display_char(i)

label: e
showing original


Prediction: e
showing the one prediction run on


label: 9
showing original


Prediction: 9
showing the one prediction run on


label: Q
showing original


Prediction: Q
showing the one prediction run on


# preprocessing


In [193]:
# read pickle

In [287]:
import pickle
# with open("test.txt", "wb") as fp:   #Pickling
#     pickle.dump(l, fp)

with open("./test/example_forms/julinka_dotaznik/example_letters.pickle", "rb") as fp:   # Unpickling
        b = pickle.load(fp)

len(b)


25

In [292]:
img = b[1]
display(PIL.Image.fromarray(img))

In [289]:
# preproces to training format

In [290]:
def display_char(img):
#     label = test_db.iloc[i].values[0]
#     char = res_mapper.get(label)ma
#     print(f"label: {char}")
    
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    
    
    img = img.astype('uint8') 
    img = img.reshape((28,28))
    img = img.transpose()
    display(PIL.Image.fromarray(img))

In [295]:
for img in b[0:]:
    display_char(revert_showing(img))

Prediction: F


Prediction: P


Prediction: P


Prediction: q


ValueError: Error when checking input: expected input_1 to have shape (784,) but got array with shape (1,)

In [183]:
# img = img.astype('uint8') 

In [184]:
pred = model.predict(np.array([img]))
print(pred)
pred_val = res_mapper.get(pred[0].argmax())
print(f"Prediction: {pred_val}")

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Prediction: J


In [ ]:
def display_char(i):
    label = test_db.iloc[i].values[0]
    char = res_mapper.get(label)
    print("=============")
    print(f"label: {char}")
    
    img = test_db.iloc[i].values[1:]
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    
    
    img = img.astype('uint8') 
    img = img.reshape((28,28))
    img = img.transpose()
    display(PIL.Image.fromarray(img))